In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#@title Importar dataframe comparación
import os
import pandas as pd


input_folder = "/content/drive/MyDrive/TFM/T3/Archivos"

df_misma_conformacion_output = os.path.join(input_folder, "df_misma_conformacion_lig.csv")
df_diff_conformacion_output = os.path.join(input_folder, "df_diff_conformacion_lig.csv")

df_misma_conformacion_lig = pd.read_csv(df_misma_conformacion_output, sep =',')
df_diff_conformacion_lig = pd.read_csv(df_diff_conformacion_output, sep =',')

In [4]:
#@title Unir dataframe comparación

df_unido = pd.concat([df_diff_conformacion_lig,df_misma_conformacion_lig])
df_unido.shape[0]

12195215

In [5]:
#@title Abrir los dataframes

import os
import pandas as pd

input_folder = "/content/drive/MyDrive/TFM/T3/Red_Neuronal/intento2"
input_file_df_final = os.path.join(input_folder, "df_FINAL_tot.csv")
df_harm = pd.read_csv(input_file_df_final, sep = ',')
print(df_harm.shape[0],df_harm.columns)

70378 Index(['PDB_entry_id', 'Classification', 'Organism', 'Uniprot_id', 'Ligand_id',
       'Ligand_InChi', 'Experimental_method', 'Resolution',
       'Adding_Classification', 'Affinity', 'Coordenadas', 'Ligand_smiles',
       'Mol_Weight', 'n_atoms', 'Nombre', 'Ligand_Class', 'Pfam_Names',
       'SUPFAM_Names', 'intepro_class'],
      dtype='object')


In [6]:
instancia_tot = df_harm['Nombre'].tolist()

In [7]:
print(df_unido.shape[0])

df_unido = df_unido[df_unido['lig1'].isin(instancia_tot)]
df_unido = df_unido[df_unido['lig2'].isin(instancia_tot)]

print(df_unido.shape[0])

12195215
12195065


In [8]:
#@title Clasificar ligandos según similitud de conformación

threshold = 0.6

# agrupamos por ligando y recogemos la serie de similitudes
grp = df_unido.groupby('ligand')['sim']

solo_misma_lig = [lig for lig, sims in grp if (sims >  threshold).all()]
solo_diff_lig  = [lig for lig, sims in grp if (sims <=  threshold).all()]

# los demás
all_ligs = df_unido['ligand'].unique()
variable_lig = [lig for lig in all_ligs
                if lig not in solo_misma_lig + solo_diff_lig]

print("solo_misma:", solo_misma_lig)
print("solo_diff: ", solo_diff_lig)
print("variable:  ", variable_lig)

solo_misma: ['010', '02A', '03M', '03V', '041', '07L', '08J', '0HK', '0HN', '0IM', '0L1', '0L9', '0PY', '0R0', '0RE', '0S6', '0SX', '0UE', '0V5', '0WK', '0X3', '0Y0', '10L', '115', '11H', '120', '123', '12D', '12Q', '132', '135', '13X', '144', '145', '146', '147', '152', '15E', '171', '173', '174', '1AB', '1AC', '1BN', '1CA', '1CY', '1DA', '1DZ', '1F1', '1FF', '1FL', '1GC', '1GH', '1HN', '1JJ', '1K0', '1KA', '1KP', '1LQ', '1MZ', '1NP', '1P8', '1PR', '1QQ', '1SQ', '1SU', '1TI', '1UN', '1XM', '1Y6', '210', '211', '21G', '22L', '22U', '23D', '23U', '241', '26D', '27H', '27J', '27L', '27M', '27N', '29L', '29N', '29T', '2AD', '2AF', '2AL', '2AM', '2AN', '2AP', '2AQ', '2AS', '2BG', '2C0', '2CH', '2CL', '2CQ', '2D0', '2EC', '2EH', '2GP', '2HA', '2HC', '2HE', '2IM', '2JJ', '2KT', '2L1', '2LX', '2LY', '2MP', '2MR', '2MZ', '2NU', '2OH', '2OP', '2PG', '2Q2', '2RH', '2SX', '2UP', '2ZM', '308', '323', '32U', '34Z', '36J', '36X', '37T', '38T', '3A9', '3AB', '3AD', '3AM', '3BZ', '3C4', '3C5', '3CH', 

Para separar los ligandos y dividir el dataset en train y test, seleccionamos:

- **train**:
  - todos los ligandos con misma conformación
  - 70% de los ligandos con conformación variable
  - 70% de los ligandos con solo conformación disimilares

- **test**:
  - 30% de los ligandos con conformación variable
  - 30% de los ligandos con solo conformación disimilares

In [9]:
#@title Separación de los ligandos

import random

random.seed(42)

lista_70_variables = random.sample(variable_lig, int(len(variable_lig) * 0.7))
print(len(lista_70_variables))

lista_70_solo_diff = random.sample(solo_diff_lig, int(len(solo_diff_lig) * 0.7))
print(len(lista_70_solo_diff))

print(len(solo_misma_lig))

train_lig = solo_misma_lig + lista_70_solo_diff + lista_70_variables
print(len(train_lig))

693
51
1807
2551


In [10]:
#@title Identificar ligandos del train y del test sets

set_30_variable = set(variable_lig) - set(lista_70_variables)
lista_30_variable = list(set_30_variable)
print(len(lista_30_variable))

set_30_solo_diff = set(solo_diff_lig) - set(lista_70_solo_diff)
lista_30_solo_diff = list(set_30_solo_diff)
print(len(lista_30_solo_diff))

test_lig = lista_30_variable + lista_30_solo_diff
print(len(test_lig))

297
23
320


In [11]:
#@title Identificar instancias (positivas) de train y test set

df_harm_train = df_harm[df_harm['Ligand_id'].isin(train_lig)]
train_instancias = df_harm_train['Nombre'].tolist()
print(len(train_instancias))

df_harm_test = df_harm[df_harm['Ligand_id'].isin(test_lig)]
test_instancias = df_harm_test['Nombre'].tolist()
print(len(test_instancias))

57310
13068


In [12]:
with open("/content/drive/MyDrive/TFM/T3/Red_Neuronal/intento_3/instancias_test_intento_13.txt", "w", encoding= "utf-8") as f:
  for line in test_instancias:
    f.write(line + "/n")

with open("/content/drive/MyDrive/TFM/T3/Red_Neuronal/intento_3/instancias_train_intento_13.txt", "w", encoding= "utf-8") as f:
  for line in train_instancias:
    f.write(line + "/n")